<a href="https://colab.research.google.com/github/GTWFam/Group7_DS3010/blob/main/CaseStudy2_MongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Study 2 : Putting Twitter Data into the Cloud

Due Date: 2/15/2020, **BEFORE the beginning of class at 2:00pm EST**

## **NOTE: There are *always* last minute issues submitting the case studies.  DO NOT WAIT UNTIL THE LAST MINUTE!**

<a title="Fabián Alexis [CC BY-SA (https://creativecommons.org/licenses/by-sa/3.0)]" href="https://commons.wikimedia.org/wiki/File:Antu_mongodb.svg"><img width="50" alt="Antu mongodb" src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f9/Antu_mongodb.svg/512px-Antu_mongodb.svg.png"></a>

**TEAM Members:** Please EDIT this cell and add the names of all the team members in your team

    Jacob Bissonette
    Austin Franklin 
    Yuriy Kamenivskyy 
    Garrett McMerriman
    Bill Hoang Pham

**Don't forget!**
* You will need to install the pymongo library to access MongoDB
 * pip install pymongo
* You will also need to install dnspython to access the cloud version of MongoDB
 * pip install dnspython

** NOTE **
* **Please don't forget to save the notebook frequently when working in IPython Notebook, otherwise the changes you made can be lost**.

# Problem 1 (20 points):  Get a cloud database account using MongoDB Atlas

Get a free cloud hosted MongoDB database account at https://www.mongodb.com/atlas/database.  You will need to: 

* Create a database cluster using the MongoDB web interface
* Get your MongoDB credentials
* Create a document collection in MongoBD


# Problem 2 (20 points):  Read Twitter data into your cloud MongoDB database

Gather Twitter data and upload it to your cloud database.  You will need to:

* Get your cloubd MongoDB credentials using the MongoDB web interface
* Insert those credentials into your Juypyter notebook using pymongo
* Gather Twitter data as in Case Study 1
* Upload your Twitter data to the MongoDB cloud document collection

Note, to do you this question remember that you will need *two sets of credentials*

* One set for the MongoDB Atlas database
* One set for the Twitter developed API

Of course, there are not the same!

In [16]:
! pip install pymongo
! pip install dnspython


In [17]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
import pymongo
import json

from google.colab import drive

drive.mount('/content/drive')

data = {}
    
# Extracting the saved credentials from the drive
with open('drive/MyDrive/ds3010_2022/credentials.txt', 'r') as f:
    data = json.load(f)
CONSUMER_KEY = data["consumer_key"]
CONSUMER_SECRET = data["consumer_secret"]
OAUTH_TOKEN = data["oauth_token"]
OAUTH_TOKEN_SECRET = data["oauth_token_secret"]
mongodb_url = data["mongodb_url"]

client = pymongo.MongoClient(mongodb_url)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
# Getting the Database and Collection from MongoDB
db = client['DS3010']
coll = db['CS2']

In [19]:
! pip install twitter

In [20]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary
import twitter

# See https://dev.twitter.com/docs/auth/oauth for more information 
# on Twitter's OAuth implementation.

auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

In [21]:
query = "macbook pro 2021"

data = twitter_api.search.tweets(q=query, lang='en', count=100)

min_id = data['statuses'][-1]['id']
for i in range(7):
    tweets = twitter_api.search.tweets(q=query, lang='en', max_id=min_id, count=100)
    min_id = tweets['statuses'][-1]['id']
    data['statuses'] += tweets['statuses']


In [22]:
statuses = data['statuses']
coll.drop()
_ = coll.insert_many(statuses)

# Problem 3 (20 points):  Use a regular expression to read a subset of your Tweets out of MongoDB

Use regular expressions to analyze properties of your data.  For example, you can 

* search for trends that have a large volume, 
* search for tweets with geotags, 
* search for tweets that can contain a certain string,
* anything else you think might be useful for a product!

In [23]:
#----------------------------------------------
# Your code starts here
#   Please add comments or text cells in between to explain the general idea of each block of the code.
#   Please feel free to add more cells below this cell if necessary

query = { "favorite_count": { "$gt": 20 } }
cursor = coll.find(query)

for tweet in cursor:
    print(tweet['text'])

Tech bro/sis, final year brethren, graphic designers, writers, anyone in need of a good laptop or money, this tweet… https://t.co/REQeY1t0YZ
Client’s MacBook Pro 14” 2021 model/16GB Ram/512GB SSD M1Pro chip - Ready 4 delivery/ More of this Still in stock… https://t.co/AzX70Jcqfh
Nah!!! Apple is Goated for their User Experience

Might fuck around and drop an extra $5K+ to pick up an iMac and u… https://t.co/22SciRqcZY
This season, @xendfinance is giving out a brand new MacBook pro 2021, and a prize pool of $2,000, to 20 savers 

To… https://t.co/8bqmdhSbYb
New saving challenge:

You save N3,000 every day from now till February 28th, and at the end, you win a MacBook Pro… https://t.co/2YFPEiTmXY
The battery life on the 2021 Macbook Pro is wild y'all... I've been using my computer for an hour &amp; it still says t… https://t.co/vtoMoLyDj5
In the spirit of love &amp; financial responsibility, @xendfinance is giving out a BRAND NEW MacBook Pro 2021, and priz… https://t.co/YPXNlAV5sZ
The 202

### There are no tweets that has a geotag

In [24]:
query2 = { "geo": { "$ne": None } }
cursor = coll.find(query2)

for tweet in cursor:
    print(tweet['text'])

In [25]:
query3 = { "text": { '$regex': '^.*cool.*$', '$options': 'si' } }
cursor = coll.find(query3)

for tweet in cursor:
    print(tweet['text'])

TECOOL Custodia per 2021 MacBook Pro 14, MacBook Air/Pro 13, 13,5 Surface Laptop, 14” Huawei Matebook, ASUS Zenbook… https://t.co/H9wSWkPrpD


In [26]:
query4 = { "text": { '$regex': '^.*battery.*$', '$options': 'si' } }
cursor = coll.find(query4)

for tweet in cursor:
    print(tweet['text'])

The battery life on the 2021 Macbook Pro is wild y'all... I've been using my computer for an hour &amp; it still says t… https://t.co/vtoMoLyDj5
RT @wccftech: Apple’s 2021 MacBook Pro Lasts 60% Longer Than the Best High-End Notebook’s Battery Timing, Claims Ne… https://t.co/FuZ58FkCog
RT @wccftech: Apple’s 2021 MacBook Pro Lasts 60% Longer Than the Best High-End Notebook’s Battery Timing, Claims New Test https://t.co/NFvB…
RT @wccftech: Apple’s 2021 MacBook Pro Lasts 60% Longer Than the Best High-End Notebook’s Battery Timing, Claims New Test https://t.co/NFvB…


# Problem 4 (20 points): Business question 

Run some additional experiments with your data to gain familiarity with the MongoDB.

* Come up with a business question that Twitter data and MongoDB could help answer.
* Decribe the business case.
* How could MongoDB help a company to scale up its computation environment?

Our business lies in helping entrepreneurs better understand their market using twitter data, prior to them making an investing decision. Our research model is flexible and can be used to research just about any topic.

The model is the following: 
1. We determine the market's underserved needs by analyzing negative sentiment tweets related to the business of choice. After collecting all negative sentiment tweets, we determine the most frequently occuring words, and then search for tweets containing these words to get an idea of what the problem / underserved need may be.
2. We determine the market's table stakes–– needs that are satisfied by all the other competitors, and thus have to be satisfied by us as well–– to do this we follow an identical algorithm to problem 1, but now we query for tweets with a positive sentiment.
3. We identify the creators of most liked and most retweeted positive tweets; and then look for overlapping accounts that they follow. These accounts will be our primary contacts for targeted advertisement of the new product.

MongoDB helps us organize this data into different collection and use regular expressions to query data efficiently and accurately.

**1. Researching Negative attitude tweets. Pushing to a MongoDB dbs. Looking for most frequent words. Looking for tweets with a geotag to see if we can determine where did negative attitudes come from. Listing 10 sentences where each of the most frequent words occurs to get the idea of what people are unhappy about.**

In [27]:
# Negative attitudes
query1 = 'burger OR burgers :( -filter:retweets'

data1 = twitter_api.search.tweets(q=query1, lang='en', count=100)

min_id = data1['statuses'][-1]['id']
for i in range(10):
    tweets = twitter_api.search.tweets(q=query1, lang='en', max_id=min_id, count=100)
    min_id = tweets['statuses'][-1]['id']
    data1['statuses'] += tweets['statuses']

statuses1 = data1['statuses']
print(len(statuses1))
coll_neg = db['cs2_coll_neg']
# coll_neg.drop()
# _ = coll_neg.insert_many(statuses1)

179


In [28]:
# Similar to Case Study 1, looking at most common words

import re
exclude = ['rt', 'in', 'the', 'for','of', 'to', 'and', 'a', 'is', 'are', 'on', 'from', 'that', 'this', 'these', 'with', 'what', 'it', 'if', 'or', 'but', 'your', 'be', 'you', 'my', 'at', 'd', 'its', 'i', 'so', 'was', 'he', 'me', 'no', 'about']

tweet_texts_neg = [ re.sub(r"[^\w\s'#@]", "", tweets['text']) for tweets in statuses1 ]
print(tweet_texts_neg[0])
words_neg = [ w for t in tweet_texts_neg for w in t.lower().split() ]
words_neg = list(filter(lambda x: x not in exclude, words_neg))
print(words_neg[1])

shadow dedede update Hammed burger 
dedede


In [29]:
from prettytable import PrettyTable
from collections import Counter

c = Counter(words_neg)
top_words_neg = c.most_common()[:10]
print(top_words_neg) # top 10

pt = PrettyTable(field_names=['Word', 'Count'])
[ pt.add_row(kv) for kv in c.most_common()[:10] ]
pt.align['Word'], pt.align['Count'] = 'l', 'r'
print(pt)

[('burger', 130), ('burgers', 28), ('like', 18), ('king', 18), ('want', 18), ('im', 17), ('dont', 15), ('had', 14), ('not', 12), ('have', 12)]
+---------+-------+
| Word    | Count |
+---------+-------+
| burger  |   130 |
| burgers |    28 |
| like    |    18 |
| king    |    18 |
| want    |    18 |
| im      |    17 |
| dont    |    15 |
| had     |    14 |
| not     |    12 |
| have    |    12 |
+---------+-------+


In [30]:
# Attempt to look for negative sentiment tweets with a geo-tag. Might give us an idea of where competitors fail or make bad burgers
query_neg1 = { "geo": { '$ne': None } }
cursor = coll_neg.find(query_neg1)

for tweet in cursor:
    print(tweet['text'])

In [31]:
# Looking at tweets that have the most used words
for combo in top_words_neg:
    print(combo[0])
    query = { "text": { '$regex': '^.* '+ combo[0] + ' .*$', '$options': 'si' } }
    cursor = coll_neg.find(query)
    i = 0
    # Printing first 10 tweets
    printed = []
    for tweet in cursor:
        # Filtering out the not useful and spamming data
        if tweet['text'] in printed:
            continue
        if i >= 10:
            break
        print(tweet['text'], '\n')
        printed.append(tweet['text'])
        i += 1

    print("-"*100)

burger
P. Terry’s got rid of their hatch green chile burger :( 

I hate my bf rn, he ate my burger I’ve been saving all night bc I was drunk :( 

shadow dedede update! Hammed burger :( 

i had a burger yest :( https://t.co/JROObe7Bke 

hobi is eating a burger i want one too :( 

Wanted to try the @JackBox Heinz Dip and Crunch Bacon Cheeseburger tonight - ended up with a dry burger and no dip… https://t.co/D75PKV7ngh 

@Harry1T6 I LOVE Burger Priest, I would have got it instead but I just had it yesterday lmao. My go to is the Doubl… https://t.co/V3s8EPmUZa 

Checking Etsy to see if anyone's made the burger shirt from the cover of Nona the Ninth yet. Can't find one. :( 

@bludsex burger king :( https://t.co/Rs1SFNwcxf 

til to melt vegan cheese on ur burger in the oven bc otherwise it will slide away :( 

----------------------------------------------------------------------------------------------------
burgers
@imnickintl I've mainly had their pasta and It was bad :( and the burgers o

**2. Researching Positive attitude tweets. Pushing to a MongoDB dbs. Looking for most frequent words. Looking for tweets with a geotag to see if we can determine where did positive attitudes come from. Listing 10 sentences where each of the most frequent words occurs to get the idea of what people are unhappy about.**

In [32]:
# Positive attitudes
query2 = 'burger OR burgers :) -filter:retweets'

data2 = twitter_api.search.tweets(q=query2, lang='en', count=100)

min_id = data2['statuses'][-1]['id']
for i in range(10):
    tweets = twitter_api.search.tweets(q=query2, lang='en', max_id=min_id, count=100)
    min_id = tweets['statuses'][-1]['id']
    data2['statuses'] += tweets['statuses']

statuses2 = data2['statuses']
print(len(statuses2))
coll_pos = db['cs2_coll_pos']
# coll_pos.drop()
# _ = coll_pos.insert_many(statuses2)

404


In [33]:
tweet_texts_pos = [ re.sub(r"[^\w\s'#@]", "", tweets['text']) for tweets in statuses2 ]
print(tweet_texts_pos[0])
words_pos = [ w for t in tweet_texts_pos for w in t.lower().split() ]
words_pos = list(filter(lambda x: x not in exclude, words_pos))
print(words_pos[1])

What are your favourite foods  Meatballs burger chicken katsu sweet and sour chicken I have too many haha but httpstcoK3xRAK5z5D
foods


In [34]:
c = Counter(words_pos)
top_words_pos = c.most_common()[:10]
print(top_words_pos) # top 10

pt = PrettyTable(field_names=['Word', 'Count'])
[ pt.add_row(kv) for kv in c.most_common()[:10] ]
pt.align['Word'], pt.align['Count'] = 'l', 'r'
print(pt)

[('burger', 244), ('burgers', 96), ('good', 33), ('get', 32), ('have', 30), ('like', 29), ('king', 28), ('just', 28), ('mob', 24), ('think', 22)]
+---------+-------+
| Word    | Count |
+---------+-------+
| burger  |   244 |
| burgers |    96 |
| good    |    33 |
| get     |    32 |
| have    |    30 |
| like    |    29 |
| king    |    28 |
| just    |    28 |
| mob     |    24 |
| think   |    22 |
+---------+-------+


In [35]:
# similar attempt to find positive "burger" tweets with a geotag
query_pos1 = { "geo": { "$ne": None } }
cursor = coll_pos.find(query_pos1)

for tweet in cursor:
    print(tweet['text'])

In [39]:
for combo in top_words_pos:
    print(combo[0])
    query = { "text": { '$regex': '^.*'+ combo[0] + '.*$', '$options': 'si' } }
    cursor = coll_pos.find(query)
    i = 0
    # Printing first 10 tweets
    for tweet in cursor:
        if tweet['text'] in printed:
            continue
        if i >= 10:
            break
        print(tweet['text'], '\n')
        printed.append(tweet['text'])
        i += 1

    print("-"*100)

burger
@ishyungshi fries and burger po from mcdo ate :) 

@billbison77 Sounds nice :) I’ll take w burger and if you need help with website i can help you build one with a takeout option 😍✌️ 

@cooolona hi po :)) i want a burger 

@kayla_eilhart Uhh. @McDonalds - Make a special burger without onion, so this lady can order anonymously next time… https://t.co/Xka28HxJdC 

shout out to whoever put the veggie burger advert right after the "eat meat and dairy :-)" advert on ITV1 

@TRIANGLE_MUPPET burger :) 

woul you rather eat pizza or ham burger :) https://t.co/tnGo3fReRH 

@Burger_Baron But don't tell him Greyhound is out of business right? :D 

@xdocvibez yes! burgers &amp; chicken tenders :) 

burger time :) 

----------------------------------------------------------------------------------------------------
burgers
@KuraiKiti This inspires hope for all burgers :) 

Follow for follow Burgers! Help you fellow burgers reach one hundred followers :D 

#TeamSehamOnKumu #KumuPh https://t.c

**3. Finding 20 most popular retweeted tweets and 20 most liked tweets from positive attitude tweets. Looking for overlaps in who these people follow. Listing these accounts as primary contacts for targeted advertisement.**

In [37]:
tweets = [
          (pop_tweet['retweet_count'], 
             pop_tweet['user']['screen_name'],
             pop_tweet['text'])
          for pop_tweet in statuses2
            if 'retweeted_status' not in pop_tweet
]
pt = PrettyTable(field_names=['RT Count', 'Screen Name', 'Text'])
[ pt.add_row(row) for row in sorted(tweets, reverse=True)[:20] ]
pt.max_width['Text'] = 50
pt.align= 'l'
print(pt)

+----------+-----------------+----------------------------------------------------+
| RT Count | Screen Name     | Text                                               |
+----------+-----------------+----------------------------------------------------+
| 30       | trioart412      | Cutie Girl Eat Burger ! :D                         |
|          |                 |                                                    |
|          |                 | Gift for @BellflowerBlu https://t.co/wwox3GPkmM    |
| 23       | tanodniseham    | solid seham burgers, don’t forget to continue      |
|          |                 | saving your coins. we still don’t know when will   |
|          |                 | seham be nominated.… https://t.co/UutMHLJ5Hv       |
| 21       | SNLSABRINA      | i need more snl and sitcom moots. so if you like   |
|          |                 | snl, b99, abbott elementary, new girl, modern      |
|          |                 | family, superstore,… https://t.co/ImIzYQuFmb 

In [38]:
tweets = [
          (pop_tweet['favorite_count'], 
             pop_tweet['user']['screen_name'],
             pop_tweet['text'])
          for pop_tweet in statuses2
            if 'retweeted_status' not in pop_tweet
]
pt = PrettyTable(field_names=['Likes Count', 'Screen Name', 'Text'])
[ pt.add_row(row) for row in sorted(tweets, reverse=True)[:20] ]
pt.max_width['Text'] = 50
pt.align= 'l'
print(pt)

+-------------+-----------------+----------------------------------------------------+
| Likes Count | Screen Name     | Text                                               |
+-------------+-----------------+----------------------------------------------------+
| 188         | trioart412      | Cutie Girl Eat Burger ! :D                         |
|             |                 |                                                    |
|             |                 | Gift for @BellflowerBlu https://t.co/wwox3GPkmM    |
| 140         | SNLSABRINA      | i need more snl and sitcom moots. so if you like   |
|             |                 | snl, b99, abbott elementary, new girl, modern      |
|             |                 | family, superstore,… https://t.co/ImIzYQuFmb       |
| 128         | Osiefish        | I gonna move my Geoguessr stream to 4 o’clock :) I |
|             |                 | need burger                                        |
| 99          | paricyte        | @meowriza

Unfortunately, upon closer examination of most liked and retweeted accounts that are related to burgers, we decided that this may be a poor tactic for determining accounts for advertisement. Unfortunately, most liked and most retweeted tweets that contain the word burger, are often related to something other than a burger.

*-----------------
# Done

All set! 

** What do you need to submit?**

* **Notebook File**: Save this IPython notebook, and find the notebook file in your folder (for example, "filename.ipynb"). This is the file you need to submit. Please make sure all the plotted tables and figures are in the notebook. If you used "ipython notebook --pylab=inline" to open the notebook, all the figures and tables should have shown up in the notebook.


* **PPT Slides**: please prepare PPT slides (for 15 minutes' talk) to present about the case study . We will ask *all* teams to present their case studies in class for this case study. 

* **Report**: please prepare a report (less than 10 pages) to report what you found in the data.
    * What data you collected? 
    * Why this topic is interesting or important to you? (Motivations)
    * How did you analyse the data?
    * What did you find in the data? 
 
     (please include figures or tables in the report, but no source code)

Please compress all the files in a zipped file.


** How to submit: **

        Please submit through canvas.wpi.edu



# Grading Criteria:

**Totoal Points: 100**


---------------------------------------------------------------------------
**Notebook results:**
    Points: 80


    -----------------------------------
    Question 1:
    Points: 20
    -----------------------------------
    
    -----------------------------------
    Question 2:
    Points: 20
    -----------------------------------
        
    -----------------------------------
    Question 3:
    Points: 20
    -----------------------------------
  
    -----------------------------------
    Question 4:  Business question
    Points: 20
    -----------------------------------


---------------------------------------------------------------------------
**Slides (for 5-10 minutes of presentation): Story-telling**
    Points: 20


1. Motivation about the data collection, why the topic is interesting to you.
    Points: 5 

2. Communicating Results (figure/table)
    Points: 10 

3. Story telling (How all the parts (data, analysis, result) fit together as a story?)
    Points: 5 
